In [1]:
import numpy as np
from collections import defaultdict

In [2]:
test0 = ['1,0,1~1,2,1',
         '0,0,2~2,0,2',
         '0,2,3~2,2,3',
         '0,0,4~0,2,4',
         '2,0,5~2,2,5',
         '0,1,6~2,1,6',
         '1,1,8~1,1,9']

test1 = ['0,0,1~0,1,1',
         '1,1,1~1,1,1',
         '0,0,2~0,0,2',
         '0,1,2~1,1,2']

test2 = ['0,0,1~1,0,1',
         '0,1,1~0,1,2',
         '0,0,5~0,0,5',
         '0,0,4~0,1,4']

data = np.genfromtxt('day22_input.txt', dtype=str, delimiter='\n', comments=None)

In [3]:
def get_blocks(data):
    blocks = []
    for line in data:
        start, end = line.split('~')
        block = []
        x, y, z = start.split(',')
        block.append([int(x), int(y), int(z)])
        x, y, z = end.split(',')
        block.append([int(x), int(y), int(z)])
        blocks.append(block)
    return np.array(blocks)

def block_position(block):
    #cell positions
    cells = []
    x0 = np.min(block[:,0])
    x1 = np.max(block[:,0])
    y0 = np.min(block[:,1])
    y1 = np.max(block[:,1])
    z0 = np.min(block[:,2])
    z1 = np.max(block[:,2])
    
    for dx in range(x0, x1+1):
        for dy in range(y0, y1+1):
            for dz in range(z0, z1+1):
                cells.append((dx,dy,dz))
                
    return np.array(cells)

def is_bottom(cells, dz):
    for c in cells:
        if c[2]-dz == 0:
            return True
    return False

def is_blocked(cells, dz, filled):
    blocked_by = []
    for c in cells:
        if (c[0],c[1],c[2]-dz) in filled:
            blocked_by.append(filled[(c[0],c[1],c[2]-dz)])
            
    return len(blocked_by)>0, np.unique(blocked_by)

def fill_volume(cells, dz, filled, idx):
    for c in cells:
        filled[(c[0],c[1],c[2]-dz)] = idx
    return filled

def complete_fall(blocks):
    filled = {}
    supports = defaultdict(list)
    supported_by = defaultdict(list)
    top = np.max(blocks[:,:,2])
    
    idxes = np.arange(0, len(blocks))
    for i in range(1, top+1):
        done = []
        for j in range(0, len(idxes)):
            idx = idxes[j]
            if np.min(blocks[idx,:,2]) == i:
                done.append(j)
                cells = block_position(blocks[idx])
                
                dz = 1
                while True:
                    blocked, b_idx = is_blocked(cells, dz, filled)
                    if is_bottom(cells, dz) or blocked:
                        filled = fill_volume(cells, dz-1, filled, idx)
                        if blocked:
                            supported_by[idx] = b_idx
                            for bi in b_idx:
                                supports[bi].append(idx)
                        break
                    dz += 1
        idxes = np.delete(idxes, done)
    return filled, supports, supported_by
                        


In [4]:
def blow(supports, supported_by, pos, destroied):
    destroied.append(pos)
    
    if len(supports[pos]) == 0:
        return destroied
    for nxt in supports[pos]:
        count = len(supported_by[nxt])
        for sb in supported_by[nxt]:
            if sb in destroied:
                count -= 1
        if count == 0:
            destroied = blow(supports, supported_by, nxt, destroied)
    return destroied
    

def run(data, prnt=False):
    blocks = get_blocks(data)
    filled, supports, supported_by = complete_fall(blocks)
    
    can_destroy = []
    for i in range(0, len(blocks)):
        num_supports = len(supports[i])
        if num_supports == 0:
            can_destroy.append(i)
        else:
            for s in supports[i]:
                if len(supported_by[s]) > 1:
                    num_supports -= 1
            if num_supports == 0:
                can_destroy.append(i) 
    can_destroy = np.unique(can_destroy)
    
    if prnt:
        abc = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E', 5:'F', 6:'G'}
        for key, value in supports.items():
            string = abc[key]+' supports'
            for v in value:
                string += ' '+abc[v]   
            print(string)
        print()

        for key, value in supported_by.items():
            string = abc[key]+' supported by'
            for v in value:
                string += ' '+abc[v]
            print(string)
        print()
    
        string = ''
        for cd in can_destroy:
            string += abc[cd]+' '
        print(string)
        print(len(can_destroy))
    else:
        print('Part 1 result:', len(can_destroy))
        
    fallen = 0
    for i in range(0, len(blocks)):
        destroied = []
        destroied = blow(supports, supported_by, i, destroied)
        fallen += len(destroied)-1
        
    if prnt:
        print()
        print(fallen)
        print()
    else:
        print('Part 2 result:', fallen)
        
run(test0, True)
#print(run(test1))
#print(run(test2))
run(data)

A supports B C
B supports D E
C supports D E
D supports F
E supports F
F supports G
G supports

B supported by A
C supported by A
D supported by B C
E supported by B C
F supported by D E
G supported by F

B C D E G 
5

7

Part 1 result: 426
Part 2 result: 61920
